# Projet 1

## Chargement des packages

In [1]:
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Chargement des bases de données

In [3]:
path = "./../projetTrain.csv"
PT   = pd.read_csv(path)

In [4]:
path = "./../projetValid.csv"
PV   = pd.read_csv(path)

## Traitement des données

In [5]:
#Ajouter la colonne Target à PV et lui assigner la valeur -1

PV['target'] = -1

In [6]:
# Merge des deux tables

P = pd.concat([PT, PV])

In [7]:
PT.shape

(51381, 128)

In [8]:
PV.shape

(8000, 128)

In [9]:
P.shape

(59381, 128)

In [10]:
P.median()

Id                     39487.000000
Product_Info_1             1.000000
Product_Info_3            26.000000
Product_Info_4             0.230769
Product_Info_5             2.000000
Product_Info_6             3.000000
Product_Info_7             1.000000
Ins_Age                    0.402985
Ht                         0.709091
Wt                         0.288703
BMI                        0.451349
Employment_Info_1          0.060000
Employment_Info_2          9.000000
Employment_Info_3          1.000000
Employment_Info_4          0.000000
Employment_Info_5          2.000000
Employment_Info_6          0.250000
InsuredInfo_1              1.000000
InsuredInfo_2              2.000000
InsuredInfo_3              6.000000
InsuredInfo_4              3.000000
InsuredInfo_5              1.000000
InsuredInfo_6              1.000000
InsuredInfo_7              1.000000
Insurance_History_1        2.000000
Insurance_History_2        1.000000
Insurance_History_3        3.000000
Insurance_History_4        2

In [11]:
P = P.fillna(P.median()) # Remplacer les NA avec les valeurs medianes des colonnes

In [12]:
train = P[P['target'] > -1] # Extraction du fichier training

In [13]:
# Dataset split

from sklearn.model_selection import train_test_split

seed = 0
test_size = 0.30
sub_train, sub_test = train_test_split(train, test_size=test_size, random_state=seed)

In [14]:
Y_train = sub_train.target.values

In [15]:
X_train = sub_train.drop(['target'], axis=1)

In [16]:
Y_test = sub_test.target.values

In [17]:
X_test = sub_test.drop(['target'], axis=1)

In [18]:
from sklearn import preprocessing

for f in train.columns:
    if train[f].dtype=='object':
        #print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        #lbl.fit(list(train[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))

## Model 1 : Random Forest

In [19]:
# 1er modèle : Random Forest

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=2, random_state=0)

clf.fit(X_train, Y_train) #training

from sklearn.metrics import accuracy_score

predictions = clf.predict(X_test) # prediction with the trained model

accuracy = accuracy_score(Y_test, predictions)

print(clf)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
Accuracy: 81.24%


## Model 2 : Decision trees

In [20]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()

clf.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score

predictions = clf.predict(X_test) # prediction with the trained model

accuracy = accuracy_score(Y_test, predictions)
print(clf)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
Accuracy: 78.44%


## Model 3 : Elastic Net

<span style="color:red">
vous ne pouvez pas calculer un r2 sur une classification !
</span>

In [21]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet(alpha=0.1, l1_ratio=0.7)

enet.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score

predictions = enet.predict(X_test) # prediction with the trained model


from sklearn.metrics import r2_score
r2_score_enet = r2_score(Y_test, predictions)

print(enet)

print("r^2 on test data : %f" % r2_score_enet)

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.7,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
r^2 on test data : 0.096159


## Model 4 : Lasso

<span style="color:red">
vous ne pouvez pas calculer un r2 sur une classification !
</span>

In [22]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)

lasso.fit(X_train, Y_train)

y_pred_lasso = lasso.predict(X_test)

r2_score_lasso = r2_score(Y_test, y_pred_lasso)
print(lasso)
print("r^2 on test data : %f" % r2_score_lasso)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
r^2 on test data : 0.066440
